In [1]:
import keras
from keras import models
from keras import layers
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('data.csv')
class_mapping = {label:idx for idx,label in
               enumerate(np.unique(data[ 'diagnosis' ] )) }
x = data.iloc[ : , 2: ]  
y = data.iloc [ : , 1].values
for i in range(len(y)):
    y[i] = class_mapping[y[i]]
x_train, x_test, y_train, y_test = train_test_split(x, y , test_size=0.1, random_state=0)

x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [2]:
class NN():
    def __init__(self, BatchSize, ActFunc, Optimizer, Loss, NeuronList):
        self.ActFunc = ActFunc
        self.Optimizer = Optimizer
        self.BatchSize = BatchSize
        self.LayerNumber = 1
        self.NeuronList = NeuronList
        self.CrossValidCount = 3
        self.Loss = Loss
        self.Chromosome = [BatchSize, ActFunc, Optimizer, Loss, NeuronList]

    def ModelFit(self, train_data, train_labels):
        network = models.Sequential()
        network.add(layers.Dense(self.NeuronList[0], activation=self.ActFunc, input_shape=(30, )))
        for i in range(1, self.LayerNumber):
            network.add(layers.Dense(self.NeuronList[i], activation=self.ActFunc))
        network.add(layers.Dense(1, activation='sigmoid'))

        network.compile(optimizer=self.Optimizer, loss = self.Loss,
                                            metrics=['acc'])
        self.history = [0 for i in range(self.CrossValidCount)]
        for i in range(self.CrossValidCount):
            self.history[i] = network.fit(train_data, train_labels, epochs=50, batch_size=self.BatchSize, verbose=0, validation_split = 0.3)
        self.EvalAcc()
        
    def EvalAcc(self):
        self.Acc = 0
        for i in range(self.CrossValidCount):
            self.Acc += max(self.history[i].history['val_acc'])
        self.Acc /= self.CrossValidCount
    
    def GetAcc(self):
        return self.Acc

In [3]:
def StartPopulation(PopulationSize, ActFuncs, Optimizers, Losses):
    Gen = []
    for i in range(PopulationSize):
        Gen.append(NN(random.randint(10, 50), random.choice(ActFuncs), random.choice(Optimizers),
                      random.choice(Losses), [random.randint(5, 15) for i in range(1)]))
    return Gen

def FitNN(a):
    for i,_ in enumerate(a):
        print(i, end = ' ')
        a[i].ModelFit(x_train, y_train)
       
    
def EvalWeights(Gen):
    Weights = []
    PopSumFF = sum([Gen[i].GetAcc() for i, _ in enumerate(Gen)])
    for i, _ in enumerate(Gen):
        Weights.append(Gen[i].GetAcc()/PopSumFF)
    return Weights


def Selection(Gen, Weights):
    parent1, parent2 = random.choices(Gen, weights = Weights, k=2)
    return [parent1, parent2]

def GenerateMask(parent):
    def rand():
        return random.randint(0,1)
    
    chromomask = []
    for gene in parent.Chromosome:
        if type(gene) != list:
            chromomask.append(rand())
        else:
            neuronmask = [rand() for _, _ in enumerate(gene)]
            chromomask.append(neuronmask)
    return chromomask
            
def GenerateChromoFromMask(parents, mask):
    child = [0, 0]
    child[0] = []
    child[1] = []
    for i, Num in enumerate(mask):
        if type(Num) != list:
            child[0].append(parents[Num].Chromosome[i])
            child[1].append(parents[Num==0].Chromosome[i])
        else:
            neurons0 = [parents[k].Chromosome[i][j] for j, k in enumerate(Num)]
            neurons1 = [parents[k==0].Chromosome[i][j] for j, k in enumerate(Num)]
            child[0].append(neurons0)
            child[1].append(neurons1)

    return (child[0], child[1])
                

def CrossOver(Gen):
    Weights = EvalWeights(Gen)
    NewGen = []
    Child = [0, 0]
    
    for n in range(len(Gen)//2):
        Parents = Selection(Gen, Weights)
        Mask = GenerateMask(Parents[0])
        ChildChromo = GenerateChromoFromMask(Parents, Mask)
        Child[0] = NN(*ChildChromo[0])
        Child[1] = NN(*ChildChromo[1])
        NewGen.append(Child[0])
        NewGen.append(Child[1])
    return NewGen

def NewPopulation(Gen, NewGen):
    Elite = 5
    for i in range(Elite):
        if NewGen[0].GetAcc()>Gen[i].GetAcc() or i==Elite:
            x=0
            for j in range(i, PopulationSize):
                Gen[j] = NewGen[x]
                x+=1
            break

def NextGen(Gen, NewGen, Elite):
    sorted(Gen, key = lambda x: x.GetAcc(), reverse = True)
    sorted(NewGen, key = lambda x: x.GetAcc(), reverse = True)
    j=0
    for i in range(Elite):
        if Gen[i].GetAcc() <= NewGen[j].GetAcc():
            Gen[i] = NewGen[j]
            j+=1
    for i in range(Elite, len(Gen)):
        Gen[i] = NewGen[j]
        j+=1            
            
            
def Mutation(NewGen, ActFuncs, Oprimizers, Losses):
    def rand():
        return random.random()

    for i in range(len(NewGen)):
        if rand()>0.9:
            if NewGen[i].BatchSize >= 10:
                NewGen[i].BatchSize += random.randint(-5, 5)
            else:
                NewGen[i].BatchSize += random.randint(1, 10)
        if rand()>0.9:
            NewGen[i].ActFunc = random.choice(ActFuncs)
        if rand()>0.9:
            NewGen[i].Loss = random.choice(Losses)
        if rand()>0.9:
            NewGen[i].Optimizer = random.choice(Oprimizers)
        for j in range(len(NewGen[i].NeuronList)):
            if rand()>0.9:
                if NewGen[i].NeuronList[j]>10:
                    deltaNeuronNumber = int(NewGen[i].NeuronList[j]/2)
                    NewGen[i].NeuronList[j] += random.randint(-deltaNeuronNumber, deltaNeuronNumber)
                else:
                    NewGen[i].NeuronList[j] += random.randint(0, 5)

def FindMean(Gen, PopulationSize):
    a = 0
    for i in Gen:
        a += i.GetAcc()
    with open('b3.txt', 'a') as inf:
        inf.write(str(a/PopulationSize)+'\n')
        
def WriteModels(Gen, GenNum):
    with open('a'+str(GenNum)+'.txt', 'w') as inf:
        for i in Gen:
            inf.write(str(i.Chromosome)+', '+str(i.GetAcc())+'\n')

In [ ]:
ActFuncs = ['sigmoid', 'tanh', 'relu', 'elu', 'selu', 'softmax', 'softplus']
Optimizers = ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam']
Losses = ['binary_crossentropy', 'kullback_leibler_divergence', 'poisson', 'mse']

with open('b3.txt', 'w') as inf:
    inf.write('')
PopSize = 20
Elite = 5
GenNum = 0

print('Generation', GenNum)
Gen = StartPopulation(PopSize, ActFuncs, Optimizers, Losses)
FitNN(Gen)
FindMean(Gen, PopSize)
WriteModels(Gen, GenNum)
print(' ')
GenNum += 1

print('Generation', GenNum)
NewGen = CrossOver(Gen)
Mutation(NewGen, ActFuncs, Optimizers, Losses)
FitNN(NewGen)
NextGen(Gen, NewGen, Elite)
FindMean(Gen, PopSize)
WriteModels(Gen, GenNum)
print(' ')
GenNum += 1
for _ in range(100):
    print('Generation', GenNum)
    NewGen = CrossOver(Gen)
    Mutation(NewGen, ActFuncs, Optimizers, Losses)
    FitNN(NewGen)
    NextGen(Gen, NewGen, Elite)
    FindMean(Gen, PopSize)
    WriteModels(Gen, GenNum)
    print(' ')
    GenNum += 1